# Healthcare Data Cleaning Notebook
## Introduction
This notebook takes the raw (but previously cleaned) healthcare datasets and applies a series of cleaning and transformation steps based on our earlier analysis. The goal is to prepare the data for loading into a database or for further analysis. Each step is broken down into a separate cell for clarity.
### Imports and Setup
First, we import the necessary libraries. pandas is for data manipulation and os is for interacting with the file system (to create directories and manage file paths).

Note: You may need to install pyarrow and sqlalchemy if you don't have them: pip install pyarrow sqlalchemy

In [ ]:
import pandas as pd
import os
import numpy as np
from sqlalchemy.dialects import postgresql

### Configuration
Design Decision: Define input and output file paths as variables in a dedicated configuration cell.
#### Why? 
This separates configuration from the core logic. Instead of hard-coding file paths deep inside the script, this design allows anyone (including your future self) to easily reuse this notebook for different data by only changing these two variables. It makes the code more maintainable and reusable.

In [3]:
INPUT_DATA_DIR = os.path.join('..', 'cleaned_data')
OUTPUT_DATA_DIR = os.path.join('..', 'v2cleaned_data')

### Load Data

Load all datasets

In [4]:
try:
    observations = pd.read_csv(os.path.join(INPUT_DATA_DIR, "observations_cleaned.csv"))
    patients = pd.read_csv(os.path.join(INPUT_DATA_DIR, "patients_cleaned.csv"))
    procedures = pd.read_csv(os.path.join(INPUT_DATA_DIR, "procedures_cleaned.csv"))
    diagnoses = pd.read_csv(os.path.join(INPUT_DATA_DIR, "diagnoses_cleaned.csv"))
    encounters = pd.read_csv(os.path.join(INPUT_DATA_DIR, "encounters_cleaned.csv"))
    medications = pd.read_csv(os.path.join(INPUT_DATA_DIR, "medications_cleaned.csv"))
    print("All datasets loaded successfully.")
except FileNotFoundError as e:
    print(f"[ERROR] A data file was not found. Please check the input directory path. Details: {e}")

All datasets loaded successfully.


Lets double check

In [5]:
observations.head()

,encounter_id,observation_code,observation_datetime,observation_description,observation_id,patient_id,units,value_numeric,value_text
0,f5f83a54-5883-413d-9bb4-c859fa6b8cde,4548-4,2025-04-14,Hemoglobin A1c/Hemoglobin.total in Blood,c70dc224-4c15-43ec-89b6-ed7821d80df2,ea3a68f6-ecf9-46aa-be97-7ecbfc7e7fcb,%,7.6,NaN
1,f5f83a54-5883-413d-9bb4-c859fa6b8cde,2345-7,2025-04-14,Glucose [Mass/Vol],065df109-6962-496e-82a7-ab975746f265,ea3a68f6-ecf9-46aa-be97-7ecbfc7e7fcb,mg/dL,210.0,NaN
2,f5f83a54-5883-413d-9bb4-c859fa6b8cde,2160-0,2025-04-14,Creatinine [Mass/Vol],ea1a0317-d4cf-4f4c-9d3b-9e87700f67bc,ea3a68f6-ecf9-46aa-be97-7ecbfc7e7fcb,mg/dL,1.0,NaN
3,a4345130-e167-45b5-9e60-75a1815d3ae0,8480-6,2026-04-08,Systolic blood pressure,8cd3eab8-0a7b-4e49-856c-ba2a081f969f,ea3a68f6-ecf9-46aa-be97-7ecbfc7e7fcb,mmHg,101.0,NaN
4,a4345130-e167-45b5-9e60-75a1815d3ae0,8462-4,2026-04-08,Diastolic blood pressure,31cb9c28-2bad-431a-b59a-6be7750e3184,ea3a68f6-ecf9-46aa-be97-7ecbfc7e7fcb,mmHg,68.0,NaN


### Cleaning the patients Table
This cell performs all cleaning and feature engineering steps for the patients data.
Design Decisions:
* Convert date_of_birth to datetime for calculations.
* Convert zip_code to str to preserve format.
* Standardize text case (.str.title(), .str.upper()) for consistency.
* Convert gender to the memory-efficient category type.
* Engineer an age column for better analysis, using a fixed date for reproducibility.

Correct data types

In [21]:
patients['date_of_birth'] = pd.to_datetime(patients['date_of_birth'], errors='coerce')
patients['zip_code'] = patients['zip_code'].astype(str)

Standardize text data

In [22]:
patients['first_name'] = patients['first_name'].str.title()
patients['last_name'] = patients['last_name'].str.title()
patients['city'] = patients['city'].str.title()
patients['state'] = patients['state'].str.upper()

Use efficient data type for gender

In [23]:
patients['gender'] = patients['gender'].astype('category')

Feature Engineering: Age

In [24]:
analysis_date = pd.to_datetime('2025-06-27')
patients['age'] = (analysis_date - patients['date_of_birth']).dt.days // 365

print("'patients' table cleaned and enhanced.")
patients.info()
print("\nSample of fully cleaned patients data:")
print(patients[['first_name', 'last_name', 'city', 'state', 'age']].head())

'patients' table cleaned and enhanced.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   address        150 non-null    object        
 1   city           150 non-null    object        
 2   date_of_birth  150 non-null    datetime64[ns]
 3   first_name     150 non-null    object        
 4   gender         150 non-null    category      
 5   last_name      150 non-null    object        
 6   patient_id     150 non-null    object        
 7   phone_number   150 non-null    object        
 8   state          150 non-null    object        
 9   zip_code       150 non-null    object        
 10  age            150 non-null    int64         
dtypes: category(1), datetime64[ns](1), int64(1), object(8)
memory usage: 12.1+ KB

Sample of fully cleaned patients data:
  first_name last_name                   city state  age
0       Juan 

### Data Validation and Feature Engineering (encounters)
Design Decisions:
* Validate that discharge_date is not before admission_date to ensure data integrity.
* Engineer a visit_duration_days column, which is more valuable for analysis than two separate date columns.
### Why?
 Logical validation prevents "impossible" data from entering the database. Feature engineering creates direct analytical value.

Convert dates to the correct type

In [25]:
encounters['admission_date'] = pd.to_datetime(encounters['admission_date'], errors='coerce')
encounters['discharge_date'] = pd.to_datetime(encounters['discharge_date'], errors='coerce')

Validation check

In [26]:
invalid_dates = encounters[encounters['discharge_date'] < encounters['admission_date']]
if not invalid_dates.empty:
    print(f"[WARNING] Found {len(invalid_dates)} encounters with discharge date before admission date.")
else:
    print("Encounter date logic is valid.")

Encounter date logic is valid.


Feature Engineering

In [27]:
encounters['visit_duration_days'] = (encounters['discharge_date'] - encounters['admission_date']).dt.days
print("'visit_duration_days' column created.")
print(encounters[['admission_date', 'discharge_date', 'visit_duration_days']].head())

'visit_duration_days' column created.
  admission_date discharge_date  visit_duration_days
0     2025-04-12     2025-04-12                    0
1     2026-04-08     2026-04-08                    0
2     2008-09-19     2008-09-19                    0
3     2008-12-14     2008-12-14                    0
4     2000-06-21     2000-06-21                    0


### Advanced Cleaning and Outlier Detection (observations)
Design Decisions:
* Detect outliers using the robust IQR method.
* Create a new boolean is_outlier column instead of deleting data.
#### Why? 
In medical data, an extreme value might be a data error or a critical event. Deleting it is risky. Flagging the data is a safer, non-destructive approach that preserves all original information while allowing analysts to easily filter if they choose.

Convert date to the correct type

In [28]:
observations['observation_datetime'] = pd.to_datetime(observations['observation_datetime'], errors='coerce')

In [29]:
def flag_outliers(df, column):
    """Flags outliers in a specified column of a DataFrame using the IQR method."""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (df[column] < lower_bound) | (df[column] > upper_bound)

Apply the function to the numeric values

In [30]:
observations['is_outlier'] = flag_outliers(observations, 'value_numeric')

CORRECTED: Fill NaN using the recommended syntax to avoid warnings.

In [31]:
observations['is_outlier'] = observations['is_outlier'].fillna(False)

outlier_count = observations['is_outlier'].sum()
print(f"Flagged {outlier_count} potential outliers in 'value_numeric'.")
print("\nSample of observations with outlier flag:")
print(observations.loc[observations['is_outlier'], ['observation_description', 'value_numeric', 'is_outlier']].head())

Flagged 9 potential outliers in 'value_numeric'.

Sample of observations with outlier flag:
    observation_description  value_numeric  is_outlier
173      Glucose [Mass/Vol]          245.0        True
205      Glucose [Mass/Vol]          249.0        True
315      Glucose [Mass/Vol]          241.0        True
416      Glucose [Mass/Vol]          248.0        True
503      Glucose [Mass/Vol]          238.0        True


### Cleaning Remaining DataFrames (procedures, diagnoses, medications)
Design Decision: Process the remaining tables, handling their specific data types and missing values as determined in our analysis.
#### Why?
 Each table has unique characteristics. We apply the specific logic needed for each one: converting dates, and for medications, correctly handling missing dosage (a quality issue) and end_date (a feature, not a bug).

In [37]:
procedures['date_performed'] = pd.to_datetime(procedures['date_performed'], errors='coerce')
diagnoses['date_recorded'] = pd.to_datetime(diagnoses['date_recorded'], errors='coerce')

medications['start_date'] = pd.to_datetime(medications['start_date'], errors='coerce')
medications['end_date'] = pd.to_datetime(medications['end_date'], errors='coerce')

Fill missing dosage with 'Unknown' - this is a data quality issue

In [38]:
medications['dosage'].fillna('Unknown', inplace=True)

C:\Users\erwin\AppData\Local\Temp\ipykernel_11248\946811579.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  medications['dosage'].fillna('Unknown', inplace=True)


CORRECTED: Check for 'Unknown' with the correct spelling.

In [39]:
print(f"Found {len(medications[medications['dosage'] == 'Unknown'])} records with dosage filled as 'Unknown'.")

Found 15 records with dosage filled as 'Unknown'.


Missing end_date means the prescription is active - we leave it as NaT

### Preparing Data for PostgreSQL
Design Decision: Save the final DataFrames as Parquet files instead of CSV.
Why? This is a critical data engineering best practice.
* CSV (The Alternative): CSV files are just plain text. They do not store metadata. When you save a DataFrame as a CSV, all data types are lost. 2025-06-27 becomes a simple string "2025-06-27". Your teammate would have to guess the correct data types, which often leads to errors.
* Parquet (Our Choice): Parquet is a modern, columnar file format designed for efficiency and data-aware systems. It saves the schema (the column names and their data types) along with the data. This makes the data handoff much more reliable.

In [47]:
dataframes = {
    "observations": observations,
    "patients": patients,
    "procedures": procedures,
    "diagnoses": diagnoses,
    "encounters": encounters,
    "medications": medications
}

for name, df in dataframes.items():
    # Trim whitespace from all object (string) columns before saving
    for col in df.select_dtypes(include=['object']).columns:
        if df[col].notna().any():
            df[col] = df[col].str.strip()

    # Define the output path
    output_path = os.path.join(OUTPUT_DATA_DIR, f"{name}.parquet")

    # Save the cleaned dataframe to a new Parquet file
    df.to_parquet(output_path, index=False)
    print(f"Successfully saved cleaned data to: {output_path}")

Successfully saved cleaned data to: ..\v2cleaned_data\observations.parquet
Successfully saved cleaned data to: ..\v2cleaned_data\patients.parquet
Successfully saved cleaned data to: ..\v2cleaned_data\procedures.parquet
Successfully saved cleaned data to: ..\v2cleaned_data\diagnoses.parquet
Successfully saved cleaned data to: ..\v2cleaned_data\encounters.parquet
Successfully saved cleaned data to: ..\v2cleaned_data\medications.parquet
